In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from target_encoding import TargetEncoderClassifier, TargetEncoder
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from functools import reduce
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.model_selection import cross_val_score
import json
from functools import partial
from dsb2019.models.coeff import ThresholdClassifier

from dsb2019.models.tracking import track_experiment, track_submission_info
from dsb2019.data.validation import InstallationFold, cross_validate, quad_kappa
from dsb2019.visualization import session_browser
from dsb2019.data import DATA_DIR
from dsb2019.models import MODELS_DIR
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import hyperopt
from hyperopt import hp, fmin, Trials, tpe, STATUS_OK
tqdm.pandas()
pd.options.display.max_rows=999

In [3]:
train = pd.read_csv(DATA_DIR / 'raw/train.csv')
test = pd.read_csv(DATA_DIR / 'raw/test.csv')
train_labels = pd.read_csv(DATA_DIR / 'raw/train_labels.csv')
submission = pd.read_csv(DATA_DIR / 'raw/sample_submission.csv')

In [4]:
games = ['Scrub-A-Dub', 'All Star Sorting', 'Mushroom Sorter (Assessment)',
       'Air Show', 'Crystals Rule', 'Bird Measurer (Assessment)',
       'Dino Drink', 'Bubble Bath', 'Dino Dive', 'Chow Time',
       'Cauldron Filler (Assessment)', 'Pan Balance', 'Happy Camel',
       'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)',
       'Leaf Leader']
worlds = ['NONE', 'MAGMAPEAK', 'TREETOPCITY', 'CRYSTALCAVES']

def unwrap_event_data(df):
    unwrapped=pd.DataFrame(data=list(df.event_data.apply(json.loads).values))
    return pd.concat([unwrapped.reset_index(),df.reset_index()],axis=1)


def process_installations(train_labels, train, process_log):
    result = []
    train=train.sort_values("timestamp")
    installations = train.groupby("installation_id")
    for i, game_session, title, installation_id, accuracy_group in tqdm(train_labels[["game_session", "title", "installation_id", "accuracy_group"]].itertuples(), 
                                                              total=len(train_labels)):
        player_log = installations.get_group(installation_id).reset_index()
        log_length = player_log[(player_log.game_session==game_session) & (player_log.title==title)].index[0]
        player_log = player_log.iloc[:(log_length + 1)]
        player_log["accuracy_group"] = accuracy_group
        player_log["target_game_session"] = game_session
        features = process_log(player_log)
        features["installation_id"] = installation_id
        features["accuracy_group"] = accuracy_group
        result.append(features)
    return pd.DataFrame(data=result)
 
    
def process_log(df):
    assessment_title=df.title.iloc[-1]   
    world=df.world.iloc[-1]

    history = df.iloc[:-1]
    history = history[history.type.isin(["Game", "Assessment"])].copy()

    def calculate_ratios(df):
        n_correct=df.correct_move.sum()
        n_incorrect=df.wrong_move.sum()
        ratio=n_correct/(n_correct+n_incorrect)
        return n_correct, n_incorrect, ratio
    
    def make_move_stats(df, title,n_lags=2):
        df=df.copy()
        if len(df):
            df = unwrap_event_data(df)
        if "correct" in df.columns:
            df["correct_move"] = df.correct == True
            df["wrong_move"] = df.correct == False
        else:
            df["correct_move"]=False
            df["wrong_move"]=False
        result = []
        result.extend(zip([f"n_correct_{title}", f"n_incorrect_{title}", f"global_ratio_{title}"], calculate_ratios(df)))

        if n_lags:
            last_sessions = df.game_session.unique()[-n_lags:]
            for i in range(n_lags):
                if i < len(last_sessions): 
                    result.extend(zip([f"n_correct_{title}_{i}", f"n_incorrect_{title} {i}",f"ratio_{title}_{i}"], 
                                      calculate_ratios(df[df.game_session==last_sessions[i]])))
                else:
                    result.extend(zip([f"n_correct_{title}_{i}", f"n_incorrect_{title}_{i}",f"ratio_{title}_{i}"], [None, None, None]))
        return {k: v for k, v in result}
    
    
    result = {"title": games.index(assessment_title),
              "world": worlds.index(world),
              "n_activities": df[df.type=="Activity"].game_session.nunique(),
              "n_games": df[df.type=="Game"].game_session.nunique(),
              "event_code_count": df.event_code.nunique(),
              "event_id_count": df.event_id.nunique(),
              "title_count": df.title.nunique(),
              "session_id_count": df.game_session.nunique(),
              "n_actions": len(df),
              "world_title_count": df[df.world==world].title.nunique(),
             }
    for game in games:
        stats=history[history.title==game]
        stats_features=make_move_stats(stats, game)
        stats_features[f"{game}_event_code_count"] = stats.event_code.nunique()
        stats_features[f"{game}_event_id_count"] = stats.event_id.nunique()
        stats_features[f"{game}_session_id_count"] = stats.game_session.nunique()
        stats_features[f"{game}_n_actions"] = len(stats)
        result.update(stats_features)
    world_games = history[history.world==world]
    for game in games:
        stats=world_games[world_games.title==game]
        stats_features=make_move_stats(stats, game)
        stats_features = {f"world_{k}": v for k, v in stats_features.items()}
        stats_features[f"world_{game}_event_code_count"] = stats.event_code.nunique()
        stats_features[f"world_{game}_event_id_count"] = stats.event_id.nunique()
        stats_features[f"world_{game}_session_id_count"] = stats.game_session.nunique()
        stats_features[f"world_{game}_n_actions"] = len(stats)
        result.update(stats_features)
    return result

In [5]:
train_features = process_installations(train_labels, train, process_log)

100%|██████████| 17690/17690 [1:47:45<00:00,  2.74it/s]  


In [6]:
train_features=train_features.fillna(-1)

In [7]:
def lgb_quad_kappa(preds, true):
    true = true.get_label()
    #preds = preds.reshape((4, -1)).argmax(axis=0)
    preds = np.rint(preds)
    preds = np.maximum(0, preds)
    preds = np.minimum(3, preds)
    return "quad_kappa", quad_kappa(true, preds), True
    
    
def train_baseline(x_train,y_train, params=None):
    x_train_all, x_val_all,y_train_all,y_val_all = train_test_split(
        x_train,y_train,
        test_size=0.15,
        random_state=2019,
    )
    train_set = lgb.Dataset(x_train_all, y_train_all)
    val_set = lgb.Dataset(x_val_all, y_val_all)

#     params = {
#         'learning_rate': 0.01,
#         'bagging_fraction': 0.9,
#         'feature_fraction': 0.9,
#         'num_leaves': 14,
#         'lambda_l1': 0.1,
#         'lambda_l2': 1,
#         'metric': 'multiclass',
#         'objective': 'multiclass',
#         'num_classes': 4,
#         'random_state': 2019
#     }

    return lgb.train(params, train_set, num_boost_round=10000, early_stopping_rounds=2000, valid_sets=[val_set], verbose_eval=100)#,
                    #feval=lgb_quad_kappa)

def make_features(df):
    return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values

def make_features_wrapper(train, test):
    def make_features(df):
        return df.drop(["installation_id", "accuracy_group"], axis=1), df.accuracy_group.values
    
    return make_features(train), make_features(test) 


def make_predictions(model,x_test_all,y_test):
    preds=model.predict(x_test_all)
    #preds = np.rint(preds)
    #preds = np.maximum(0, preds)
    #preds = np.minimum(3, preds)
    return preds,y_test

In [8]:
def process_test_installations(test):
    test = test.sort_values("timestamp")
    test=test.groupby("installation_id").progress_apply(process_log).reset_index()
    test.columns = ["installation_id", "features"]
    result = []
    for i, installation_id, feature in test.itertuples():
        result.append(feature)
        feature["installation_id"]=installation_id
    return pd.DataFrame(result).fillna(-1)
test_features=process_test_installations(test)

100%|██████████| 1000/1000 [04:18<00:00,  3.87it/s]


In [9]:
subtrain_installations=pd.Series(train_features.installation_id.unique()).sample(frac=1., random_state=2019)
subtrain_features=train_features[train_features.installation_id.isin(subtrain_installations.values)].copy()
def check_hyperparams(params):
    print(params)
    if "max_depth" in params:
        params["max_depth"] = int(params["max_depth"])
    if "num_leaves" in params:
        params["num_leaves"] = int(params["num_leaves"])

    train_baseline_with_params = partial(train_baseline, params=params)
    cv=InstallationFold(n_splits=3)
    predictions = cross_validate(subtrain_features, subtrain_features.accuracy_group, make_features_wrapper, train_baseline_with_params, make_predictions,
                                cv=cv)
    return {
        "loss": np.mean([mean_squared_error(true, pred) for pred, true in predictions]),
        "status": STATUS_OK,
        "params": params
    }


def tune(check_params, n_tries=25, n_learning_rate_tries=15, learning_rate=None, n_estimators=10_000):        
    if learning_rate is None:
        learning_rate_space = {
            'learning_rate': hp.loguniform("learning_rate", np.log(0.005), np.log(0.3)),
            'metric': 'rmse',
            'objective': 'rmse',
            #'num_classes': 4,
            'random_state': 2019,
            "n_estimators": n_estimators,

        }
        trials = Trials()
        result = fmin(check_params,
                      learning_rate_space, tpe.suggest, n_learning_rate_tries, trials=trials)
        print(result)
        learning_rate = round(trials.best_trial["result"]["params"]["learning_rate"], 3)

    param_space = {
        'metric': 'rmse',
        'objective': 'rmse',
        #'num_classes': 4,
        'lambda_l1': hp.uniform("lamba_l1", 1e-10, 1),
        'lambda_l2': hp.uniform("lambda_l2", 1e-10, 1),
        'random_state': 2019,
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": hp.quniform("max_depth", 2, 16, 1),
        "num_leaves": hp.choice("num_leaves", [3, 7, 15, 31, 63, 127, 255, 511, 1023, 2047, 4095]),
        "subsample": hp.quniform("subsample", 0.01, 1, 0.01),
        "feature_fraction": hp.quniform("feature_fraction", 0.01, 1, 0.01),
    }

    trials = Trials()
    fmin(check_params,
         param_space, tpe.suggest, n_tries, trials=trials)
    best_params = trials.best_trial["result"]["params"]
    return best_params

In [ ]:
best_params=tune(check_hyperparams, n_tries=100, n_learning_rate_tries=10)

{'learning_rate': 0.012685947867548803, 'metric': 'rmse', 'n_estimators': 10000, 'objective': 'rmse', 'random_state': 2019}
  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.04205                       
[200]	valid_0's rmse: 1.00562                       
[300]	valid_0's rmse: 0.995821                      
[400]	valid_0's rmse: 0.993108                      
[500]	valid_0's rmse: 0.991813                      
[600]	valid_0's rmse: 0.991156                      
[700]	valid_0's rmse: 0.991417                      
[800]	valid_0's rmse: 0.992229                      
[900]	valid_0's rmse: 0.992602                      
[1000]	valid_0's rmse: 0.993344                     
[1100]	valid_0's rmse: 0.993494                     
[1200]	valid_0's rmse: 0.994501                     
[1300]	valid_0's rmse: 0.99537                      
[1400]	valid_0's rmse: 0.996507                     
[1500]	valid_0's rmse: 0.997912                     
[1600]	valid_0's rmse: 0.998898                     
[1700]	valid_0's rmse: 0.999963                     
[1800]	valid_0's rmse: 1.00102      

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 2000 rounds               
[100]	valid_0's rmse: 1.0464                                                 
[200]	valid_0's rmse: 1.00909                                                
[300]	valid_0's rmse: 0.997717                                               
[400]	valid_0's rmse: 0.994341                                               
[500]	valid_0's rmse: 0.99351                                                
[600]	valid_0's rmse: 0.992529                                               
[700]	valid_0's rmse: 0.992482                                               
[800]	valid_0's rmse: 0.992521                                               
[900]	valid_0's rmse: 0.993419                                               
[1000]	valid_0's rmse: 0.994465                                              
[1100]	valid_0's rmse: 0.995749                                              
[1200]	valid_0's rmse: 0.996761                                 

...

Training until validation scores don't improve for 2000 rounds                   
[100]	valid_0's rmse: 1.11867                                                    
[200]	valid_0's rmse: 1.06171                                                    
[300]	valid_0's rmse: 1.03826                                                    
[400]	valid_0's rmse: 1.01853                                                    
[500]	valid_0's rmse: 1.01385                                                    
[600]	valid_0's rmse: 1.00949                                                    
[700]	valid_0's rmse: 1.0068                                                     
[800]	valid_0's rmse: 1.00542                                                    
[900]	valid_0's rmse: 1.00468                                                    
[1000]	valid_0's rmse: 1.00438                                                   
[1100]	valid_0's rmse: 1.00465                                                   
[1200]	valid_0's

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 2000 rounds                   
[100]	valid_0's rmse: 1.10849                                                    
[200]	valid_0's rmse: 1.06135                                                    
[300]	valid_0's rmse: 1.03853                                                    
[400]	valid_0's rmse: 1.02617                                                    
[500]	valid_0's rmse: 1.01879                                                    
[600]	valid_0's rmse: 1.01485                                                    
[700]	valid_0's rmse: 1.01264                                                    
[800]	valid_0's rmse: 1.01144                                                    
[900]	valid_0's rmse: 1.01019                                                    
[1000]	valid_0's rmse: 1.00906                                                   
[1100]	valid_0's rmse: 1.0081                                                    
[1200]	valid_0's

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



Training until validation scores don't improve for 2000 rounds                   
[100]	valid_0's rmse: 1.0394                                                     
[200]	valid_0's rmse: 1.00843                                                    
[300]	valid_0's rmse: 1.00492                                                    
[400]	valid_0's rmse: 1.00487                                                    
[500]	valid_0's rmse: 1.00611                                                    
[600]	valid_0's rmse: 1.00737                                                    
[700]	valid_0's rmse: 1.00923                                                    
[800]	valid_0's rmse: 1.0106                                                     
[900]	valid_0's rmse: 1.01176                                                    
[1000]	valid_0's rmse: 1.01325                                                   
[1100]	valid_0's rmse: 1.01436                                                   
[1200]	valid_0's

In [19]:
best_params

{'feature_fraction': 0.52,
 'lambda_l1': 0.35160015279139745,
 'lambda_l2': 0.5766835310965762,
 'learning_rate': 0.012,
 'max_depth': 15,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 31,
 'objective': 'rmse',
 'random_state': 2019,
 'subsample': 0.32}

What was selected on 100% of the data

```
{'feature_fraction': 0.44,
 'lambda_l1': 0.2342366019673217,
 'lambda_l2': 0.20913848894987938,
 'learning_rate': 0.04,
 'max_depth': 9,
 'metric': 'rmse',
 'n_estimators': 10000,
 'num_leaves': 31,
 'objective': 'rmse',
 'random_state': 2019,
 'subsample': 0.86}
```

In [20]:
# best_params={'feature_fraction': 0.58,
#  'lambda_l1': 0.45619796864269707,
#  'lambda_l2': 0.033257384218246686,
#  'learning_rate': 0.007,
#  'max_depth': 14,
#  'metric': 'multiclass',
#  'n_estimators': 10000,
#  'num_classes': 4,
#  'num_leaves': 31,
#  'objective': 'multiclass',
#  'random_state': 2019,
#  'subsample': 0.9500000000000001}

In [27]:
with open("../dsb2019/models/regression_baseline_round_params.json", "w") as f:
    json.dump(best_params, f)

In [28]:
baseline_model=train_baseline(train_features.drop(["installation_id", "accuracy_group"], axis=1), train_features.accuracy_group.values, 
               params=best_params)

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.05766
[200]	valid_0's rmse: 1.00271
[300]	valid_0's rmse: 0.985403
[400]	valid_0's rmse: 0.979708
[500]	valid_0's rmse: 0.977786
[600]	valid_0's rmse: 0.976314
[700]	valid_0's rmse: 0.976154
[800]	valid_0's rmse: 0.975923
[900]	valid_0's rmse: 0.975771
[1000]	valid_0's rmse: 0.97578
[1100]	valid_0's rmse: 0.976045
[1200]	valid_0's rmse: 0.976377
[1300]	valid_0's rmse: 0.977028
[1400]	valid_0's rmse: 0.977133
[1500]	valid_0's rmse: 0.977185
[1600]	valid_0's rmse: 0.977347
[1700]	valid_0's rmse: 0.977657
[1800]	valid_0's rmse: 0.978165
[1900]	valid_0's rmse: 0.978677
[2000]	valid_0's rmse: 0.979032
[2100]	valid_0's rmse: 0.979203
[2200]	valid_0's rmse: 0.980088
[2300]	valid_0's rmse: 0.980672
[2400]	valid_0's rmse: 0.981096
[2500]	valid_0's rmse: 0.981787
[2600]	valid_0's rmse: 0.982283
[2700]	valid_0's rmse: 0.983052
[2800]	valid_0's rmse: 0.983749
[2900]	valid_0's rmse: 0.984426
Early stopping, best 

In [29]:
predictions = cross_validate(train_features, train_features.accuracy_group, make_features_wrapper, partial(train_baseline, params=best_params), 
                             make_predictions)
np.mean([mean_squared_error(true, pred) for pred, true in predictions]), [mean_squared_error(true, pred) for pred, true in predictions]

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 2000 rounds
[100]	valid_0's rmse: 1.06459
[200]	valid_0's rmse: 1.01318
[300]	valid_0's rmse: 1.00061
[400]	valid_0's rmse: 0.995921
[500]	valid_0's rmse: 0.994159
[600]	valid_0's rmse: 0.993463
[700]	valid_0's rmse: 0.993005
[800]	valid_0's rmse: 0.993177
[900]	valid_0's rmse: 0.993457
[1000]	valid_0's rmse: 0.994023
[1100]	valid_0's rmse: 0.99505
[1200]	valid_0's rmse: 0.995983
[1300]	valid_0's rmse: 0.996636
[1400]	valid_0's rmse: 0.997398
[1500]	valid_0's rmse: 0.998195
[1600]	valid_0's rmse: 0.998994
[1700]	valid_0's rmse: 0.999715
[1800]	valid_0's rmse: 1.00037
[1900]	valid_0's rmse: 1.0012
[2000]	valid_0's rmse: 1.00226
[2100]	valid_0's rmse: 1.0032
[2200]	valid_0's rmse: 1.00393
[2300]	valid_0's rmse: 1.00464
[2400]	valid_0's rmse: 1.0054
[2500]	valid_0's rmse: 1.00587
[2600]	valid_0's rmse: 1.00644
Early stopping, best iteration is:
[665]	valid_0's rmse: 0.992944
Training until validation scores don't improve for 2000 rounds
[

(0.9916973683137508,
 [0.958425549529084,
  0.9823449245124151,
  0.9543080940105404,
  1.0184801519970772,
  1.0449281215196373])

In [30]:
baseline_model.save_model(str(MODELS_DIR / "regression_baseline_round.lgb"))

In [31]:
features, target = make_features(train_features)
prediction=baseline_model.predict(features)
clf = ThresholdClassifier()
clf.fit(prediction, target)

100%|██████████| 1000/1000 [00:23<00:00, 42.81it/s, best loss: -0.7244782390446345]


ThresholdClassifier(n_iter=1000, random_state=2019)

In [32]:
clf.coef_

[1.1602007006180572, 1.684586995754729, 2.247186521630739]